In [ ]:
"""
CS 484 Final Project

Alexander Wei, a6wei@uwaterloo.ca, 20836214
Kaiz Nanji

Project #5
Semi-supervised image classification: assuming that only M out of N images in the traingin data have ground truth labels,
design and implement a weakly supervsied training of classification network that can benefit from unlabeled examples in
the training dataset(e.g. MNIST or CIFAR-10, but you need to ignore labels on a subset of training examples).You should
demonstrate how the performance changes as M gets progressively smaller. While you can use any well-motivated ideas, one
basic approach could be to combine cross-entropy on labeled points with (unsupervised) K-means clustering loss over deep
features (e.g. in the last layer before the linear classifier). It is also advisable to use augmentation (a loss
enforcing consistent labeling of augmented training examples). You can also explore Mutual Information loss function
formulated in Bridle & MacKay "Unsupervised Classifiers, Mutual Information and Phantom Targets", NIPS 1991. 
"""

In [ ]:
from src.model import *
from src.utils import *

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
print(torch.cuda.device_count())

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
"""
Checks the filepath "./data/MNIST/raw" for the dataset. If not found, downloads the dataset
"""
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,), (0.3081,))
])

mnist_train, mnist_test = download(transform)


In [ ]:
# Increase TRAIN_BATCH_SIZE if you are using GPU to speed up training. 
# When batch size changes, the learning rate may also need to be adjusted. 
# Note that batch size maybe limited by your GPU memory, so adjust if you get "run out of GPU memory" error.
TRAIN_BATCH_SIZE = 100

# If you are NOT using Windows, set NUM_WORKERS to anything you want, e.g. NUM_WORKERS = 4,
# but Windows has issues with multi-process dataloaders, so NUM_WORKERS must be 0 for Windows.
NUM_WORKERS = 0


train_loader = DataLoader(mnist_train, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
val_loader = DataLoader(mnist_test, batch_size=1, num_workers=NUM_WORKERS, shuffle=False)

In [ ]:
my_gmm = MyGaussianMixture()

my_loss_function = MyLossFunction(my_gmm)
my_loss_function = my_loss_function.to(device)

my_model = MyModel(my_loss_function, my_gmm)
my_model = my_model.to(device)

sgd_optimizer = get_optimizer(my_model)

In [ ]:
loss_graph = []

train_model(my_model, device, train_loader, sgd_optimizer, loss_graph)